# Amazon Product Reviews Analysis

Import libraries 

In [1]:
# Import Required Libraries
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import plotly.express as px

Read data